In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn import svm
import tensorflow as tf
from tabulate import tabulate


import sys 
import os
sys.path.append('/'.join(os.getcwd().split('/')[:4]))
from config.get import cfg

2022-01-13 08:37:08.494220: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2


# Loading the data
Predictions will use embeddings produced by the previously selected embeddings model as features. The target variable of the prediction is the boolean value corresponding to the profitability of cycles.

In [2]:
# load the features
X_train = np.load(cfg['files']["liquid"]['encoded_train_features'])
X_test  = np.load(cfg['files']["liquid"]['encoded_test_features'])

In [3]:
fX_train = pd.read_csv(cfg['files']["liquid"]['additional_features_train'])
fX_test  = pd.read_csv(cfg['files']["liquid"]['additional_features_test'])

In [4]:
y_train = fX_train.profitability
y_test  = fX_test.profitability

In [5]:
print(y_train.mean()) # imbalanced classes

0.9554704815635704


In [6]:
print(y_test.mean()) # imbalanced classes

0.9564867042707494


# Motivation


The goal of the project also consists of testing the predictability of the cycle's profitability. The return of a given cycle is defined by its `revenues` minus its `cost` (fees). `Profitability` is a Boolean value indicating if the corresponding cycle has positive or negative `profitability`. `Profitability` is then used as a target/label for classification tasks. 95% of the cycles have a positive return. This imbalance can badly affect the training process. Indeed, a model that always outputs true will obtain a precision of 94% despite being meaningless. We proposed to deal with the imbalance through `class weights` in the loss function. The idea is to reweight the samples during training to obtain a 1 to 1 balance between positive and negative data points. 

Obviously, at first, the models will make use of embeddings produced by the encoding layer of the autoencoders as features. Multiple embeddings are tested (`AE`, `PCA`)

# Rescale the features
Once again, we need to normalise the encoded embeddings features

In [7]:
def scaling(X_train, X_test):
    scaler = StandardScaler()
    scaler.fit(X_train)
    tX_train = scaler.transform(X_train)
    tX_test  = scaler.transform(X_test)
    return tX_train, tX_test

In [8]:
tX_train, tX_test = scaling(X_train, X_test)

# Transform train / test into Pandas

In the following section (token one hot encoding), we will need a pandas representation of the data

In [9]:
pX_train = pd.DataFrame(data=tX_train, columns=[str(c) for c in range(X_train.shape[1])])
pX_test  = pd.DataFrame(data=tX_test,  columns=[str(c) for c in range(X_train.shape[1])])

# Token one hot encoding

In our initial dataset, we also have access to the names of the 3 tokens participating in the cyclic arbitrage, which could potentially be used as extra features! However, machine learning models usually don't like strings features. Let's tokenize them! Since we are dealing with a fixed (categorical) set of non-ordered features, a `one-hot` encoding is probably a good way to go. 

For instance, imagine we only have 3 tokens in our dataset : 

> `ETH`, `DAI` and `AAVE`

Then one could use the following `one-hot` encoding to represent them. We have 3 tokens so the encoding will be 3-dimensional 

| Token Name | \| | Dim 1 | Dim 2 | Dim 3  |
|:----------:|:--:|:-----:|:-----:|:------:|
| `ETH`      | \| |  1    |   0   |   0    |
| `DAI`      | \| |  0    |   1   |   0    |
| `AAVE`     | \| |  0    |   0   |   1    |


From a linear algebra perspective, we observe that all rows have the same norm and are linearly independent, this is what makes this `one-hot` encoding an excellent choice for our purposes.

We draw the attention of the reader to the fact that these extra features should not be added as an input to the convolutional autoencoder. Indeed, there is no translation bias to exploit here. In order to ease the performance comparison with other types of embedding models, we decided not to use them in any of the embedding related tasks. 

To sum up, two different sets of features are used as input for prediction: 
1. `Embeddings`.
2. `Embeddings + tokens`.

These two types of features are used and the scores are compared to see if the names of the involved tokens bring relevant information to the prediction. 

In [10]:
from sklearn.preprocessing import OneHotEncoder

class TokenEncoding:
    def __init__(self):
        self.one_enc = OneHotEncoder(sparse=False, handle_unknown = 'ignore') 
        self.token_columns = ['token1','token2', 'token3']

    def fit_tokens(self,data):
        unique_tokens = np.unique(pd.concat([data[token] for token in self.token_columns],axis=0))
        self.one_enc.fit(unique_tokens.reshape(-1, 1))
        return self

    def transform_tokens(self,data):
        # transform
        encode = lambda col: self.one_enc.transform(data[col].to_numpy().reshape(-1, 1))
        # encode and convert as dataframes
        encoded = [pd.DataFrame(encode(token)).add_prefix(f"token_{key}_") for key,token in enumerate(self.token_columns)]
        return pd.concat(encoded, axis='columns').astype('float32')
    

    def join_tokens(self,data, tokens):
        if ('token1' in data.columns):
            data = data.drop(columns=self.token_columns)
        return pd.concat([data,tokens], axis='columns')

In [11]:
tokenEnc = TokenEncoding()
tokenEnc.fit_tokens(fX_train)
fpX_train = tokenEnc.join_tokens(pX_train, tokenEnc.transform_tokens(fX_train))
fpX_test  = tokenEnc.join_tokens(pX_test, tokenEnc.transform_tokens(fX_test))

In [12]:
fpX_train.head()

,0,1,2,3,4,5,6,7,8,9,...,token_2_82,token_2_83,token_2_84,token_2_85,token_2_86,token_2_87,token_2_88,token_2_89,token_2_90,token_2_91
0,0.675278,2.831606,1.971766,1.117053,-0.493673,-0.628367,-0.031847,-0.279291,0.193998,-0.083904,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.645527,-0.676287,0.436694,-0.694777,0.126385,0.596609,-0.735012,1.395763,-0.049524,-0.692553,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.704034,-0.683975,-0.714685,1.435156,0.480271,1.294144,-0.018415,-0.783703,-0.695037,0.253102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.713381,-0.684020,-0.717902,1.479488,0.482580,1.284424,-0.017901,-0.777837,-0.695419,0.261080,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.895074,0.638673,-0.738661,-0.089209,-0.526641,0.294607,-0.821443,1.466609,-0.564274,2.479410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Logistic regression

## model creation & fitting

In [11]:
def fit_logistic(X_train, y_train):
    logistic_model = LogisticRegressionCV(
        cv=5,
        Cs=np.logspace(-4,4,10),
        max_iter=200,
        class_weight="balanced", 
        verbose=1
    )
    logistic_model.fit(X_train, y_train)
    return logistic_model

In [12]:
logistic_model = fit_logistic(tX_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          101     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.75341D+03    |proj g|=  2.48218D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  101     10     13      1     0     0   4.363D-03   2.724D+03
  F =   2724.2319653109762     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          101     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.7

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  101     40     45      1     0     0   2.059D-02   2.371D+03
  F =   2371.4571886948593     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          101     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.36578D+03    |proj g|=  1.39609D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


...

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  101     15     17      1     0     0   1.526D-02   2.385D+03
  F =   2385.0138579225240     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          101     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.38493D+03    |proj g|=  3.28980D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  101     34     38      1     0     0   3.579D-02   2.383D+03
  F =   2383.2494532981927     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          101     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.35366D+03    |proj g|=  8.29107D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.5s finished


## Model evaluation


Finally, we need to select a performance metric to compare our models. Usually, for binary classification tasks, the `accuracy is chosen. However, it is probably not a good chose here since we have a strong class imbalance. Indeed, using `accuracy the trivial model that outputs always true will have a high performance and this is not what we want. Hence, we will investigate the differences in terms of `precision` and `recall`, in particular through the `f1-score` metric which is a geometric mean between `precision` and `recall`.

In [14]:
def print_confusion(tn, fp, fn, tp):
    print(f"True neg : {tn} | False pos : {fp} | False neg : {fn} | True pos : {tp}")
    print(tabulate([['True (real)',tp, fn], ['False (Real)',fp, tn]], headers=['\\', 'True (pred)' ," False (pred)"], tablefmt='fancy_grid'))
   
def evaluate_model(model,test_set):

    pred = model.predict(test_set)>0.5
    tn, fp, fn, tp = confusion_matrix(y_test,pred).ravel()
    print_confusion(tn, fp, fn, tp)
    f1 = f1_score(y_test,pred)
    print(f"f1 score={f1:0.4f}")

In [15]:
evaluate_model(logistic_model, test_set=X_test)

True neg : 29 | False pos : 25 | False neg : 530 | True pos : 657
╒══════════════╤═══════════════╤═════════════════╕
│ \            │   True (pred) │    False (pred) │
╞══════════════╪═══════════════╪═════════════════╡
│ True (real)  │           657 │             530 │
├──────────────┼───────────────┼─────────────────┤
│ False (Real) │            25 │              29 │
╘══════════════╧═══════════════╧═════════════════╛
f1 score=0.7030


## Let's add the token encoding

In [17]:
logistic_model = fit_logistic(fpX_train, y_train)
evaluate_model(logistic_model, test_set=fpX_test)

True neg : 18 | False pos : 36 | False neg : 358 | True pos : 829
╒══════════════╤═══════════════╤═════════════════╕
│ \            │   True (pred) │    False (pred) │
╞══════════════╪═══════════════╪═════════════════╡
│ True (real)  │           829 │             358 │
├──────────────┼───────────────┼─────────────────┤
│ False (Real) │            36 │              18 │
╘══════════════╧═══════════════╧═════════════════╛
f1 score=0.8080


# SVM

## Model creation

In [6]:
svm_parameters = {'kernel':('linear', 'rbf','poly'), 'C':np.logspace(-4,4,5)}
svc = svm.SVC()
svm_model = GridSearchCV(svc, svm_parameters,verbose=1,cv = 2)

## Fitting the model

In [ ]:
svm_model.fit(tX_train, y_train)

Fitting 2 folds for each of 15 candidates, totalling 30 fits


GridSearchCV(cv=2, estimator=SVC(class_weight='balanced'),
             param_grid={'C': array([1.e-04, 1.e-02, 1.e+00, 1.e+02, 1.e+04]),
                         'kernel': ('linear', 'rbf', 'poly')},
             verbose=1)

## Model evaluation

In [ ]:
evaluate_model(svm_model)

True neg : 0 | False pos : 49 | False neg : 0 | True pos : 1192
╒══════════════╤═══════════════╤═════════════════╕
│ \            │   True (pred) │    False (pred) │
╞══════════════╪═══════════════╪═════════════════╡
│ True (real)  │          1192 │               0 │
├──────────────┼───────────────┼─────────────────┤
│ False (Real) │            49 │               0 │
╘══════════════╧═══════════════╧═════════════════╛
f1 score=0.9799


# Investigation of the different embeddings performance

To evaluate the performance of our cycle embedding (autoencoder), we propose to study the impact of different embeddings features on the output of a binary classification task, precisely the profitability of a cycle.

The emphasis is not put on finding the best overall model here. The idea is to study the difference in the confusion matrix and metrics (accuracy, f1-score, recall, precision) that occur when the input features changes.

If we observe better performance metrics for our autoencoder embedding, this experiment will have provided evidence that our embedding somehow captures the underlying structure of cyclic arbitrages. 


## Rule-based Embedding
In this section we are going to use a ruled based embedding ([see README in embedding folder](../embedding/rule_based/README.md)) for more details about the indicators used) to compared the preformance of our ae embedding.

In [16]:
bX_train = np.load(cfg['files']['liquid']['rule_based']['scaled_encoded_train_features'])
bX_test  = np.load(cfg['files']['liquid']['rule_based']['scaled_encoded_test_features'])

In [17]:
# reshape data
bX_train = bX_train.reshape((bX_train.shape[0],-1))
bX_test = bX_test.reshape((bX_test.shape[0],-1))

In [18]:
tbX_train, tbX_test = scaling(bX_train,bX_test)

In [19]:
# note : running the following cell takes about 2h to run 
logistic_model = fit_logistic(tbX_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        32401     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.75341D+03    |proj g|=  2.79942D+02


 This problem is unconstrained.



At iterate   50    f=  2.20699D+03    |proj g|=  3.27019D+01

At iterate  100    f=  2.20377D+03    |proj g|=  9.46650D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
32401    101    112      1     0     0   1.648D-01   2.204D+03
  F =   2203.7697414238819     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.


...

/home/lgiordan/dex-cyclic-arbitrage/opt/venv-gcc/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
 This problem is unconstrained.



At iterate   50    f=  1.00562D+01    |proj g|=  1.10735D-01

At iterate  100    f=  9.86691D+00    |proj g|=  1.95290D-01

At iterate  150    f=  9.74536D+00    |proj g|=  8.08151D-02

At iterate  200    f=  9.62288D+00    |proj g|=  6.75278D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
32401    200    216      1     0     0   6.753D-02   9.623D+00
  F =   9.6228809694513409     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        32401     M =           10

At X0         0 variables are exactly at the bounds

At iter

/home/lgiordan/dex-cyclic-arbitrage/opt/venv-gcc/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
 This problem is unconstrained.



At iterate   50    f=  9.29687D+00    |proj g|=  7.41959D-02

At iterate  100    f=  9.25394D+00    |proj g|=  7.64646D-02

At iterate  150    f=  9.19926D+00    |proj g|=  8.00623D-02

At iterate  200    f=  9.12828D+00    |proj g|=  8.86019D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
32401    200    226      1     0     0   8.860D-02   9.128D+00
  F =   9.1282760361566702     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


/home/lgiordan/dex-cyclic-arbitrage/opt/venv-gcc/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 30.2min finished
/home/lgiordan/dex-cyclic-arbitrage/opt/venv-gcc/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please 

In [21]:
evaluate_model(logistic_model, test_set=tbX_test)

True neg : 12 | False pos : 42 | False neg : 210 | True pos : 977
╒══════════════╤═══════════════╤═════════════════╕
│ \            │   True (pred) │    False (pred) │
╞══════════════╪═══════════════╪═════════════════╡
│ True (real)  │           977 │             210 │
├──────────────┼───────────────┼─────────────────┤
│ False (Real) │            42 │              12 │
╘══════════════╧═══════════════╧═════════════════╛
f1 score=0.8858


## PCA Embedding


In [ ]:
pca_X_train = np.load(cfg['files']["liquid"]["pca"]["encoded_train_features"])
pca_X_test  = np.load(cfg['files']["liquid"]["pca"]["encoded_test_features"])

In [ ]:
pca_tX_train, pca_tX_test = scaling(pca_X_train,pca_X_test)

In [ ]:
logistic_model = fit_logistic(pca_tX_train, y_train)

In [ ]:
evaluate_model(logistic_model, test_set=pca_tX_test)